1. Assume that we have 25 base learners, where each one of the learners have error rate of e = 0.35. Also assume that we use averaging as the ensemble method. If all classifiers are independent of each other, what is the probability that ensemble of these learners makes a wrong prediction?

In [0]:
import numpy as np
import pandas as pd
from scipy.stats import binom

In [18]:
#we take the binomial distribution between (25 and 25 //2 with the error rate of 0.35)

wrong_pred = 0.0
for i in range(13,25):
  wrong_pred += binom.pmf(i,25,0.35)

wrong_pred

0.06044491356302354

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/DS Grad School/DS8004/Project 3/')

In [0]:
ls

ai_soln_q3.docx           Assignment3_Leo.ipynb  Q4_leo.ipynb
ai_soln_question_2.ipynb  drink_quality.csv



2. (30 pts) Consider the drink quality classification problem over the dataset drink quality.csv (see D2L), which is posed as a multiclass classification task.

(a.) Implement a bagging classifier with Logistic Regression (use sklearn library) as the base learner and 10 estimators. Note that the predictions of the ensemble method are made based on plurality vote.

• Report the prediction accuracy over an arbitrarily selected test set.

• Compare the performance of your model with sklearn library function BaggingClassifier.

In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv('drink_quality.csv')
train, test = train_test_split(df, test_size=0.2)

In [0]:
X_test = test.iloc[:,0:10]
Y_test = test['quality']

In [0]:
#our log reg model
clf = LogisticRegression(multi_class='ovr')

In [0]:
#for each learner I will sample approximately, 63% accordinging to wiki
#https://en.wikipedia.org/wiki/Bootstrap_aggregating
res = []

for i in range(10):
  bag_data = train.sample(frac = 0.63, replace=True)
  bag_train = bag_data.iloc[:,0:10]
  bag_label = bag_data['quality']
  model = clf.fit(bag_train, bag_label)
  y_pred = model.predict(test.iloc[:,0:10])
  res.append(y_pred)

In [0]:
#we need to take the mode of each of our results
a = []

for j in range(140):
  for i in range(10):
    a.append(res[i][j])

#reshape my results
b = np.array(a)
c = b.reshape((140,10))

In [0]:
from scipy import stats

bag_pred = []
for i in range(140):
  bag_pred.append(stats.mode(c[i])[0][0])

In [0]:
sum(bag_pred == test['quality'])

77

In [0]:
#compare with sklearn 

from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(base_estimator=LogisticRegression(multi_class='ovr'), n_estimators=10, random_state=0).fit(train.iloc[:,0:10],train['quality'])

sk_pred = bag_clf.predict(test.iloc[:,0:10])

In [0]:
sum(sk_pred == test['quality']) / len(test['quality'])
#slightly better than ours

0.5785714285714286

(b.) Implement a voting classifier with 3 base learners from sklearn library: LogisticRegression, Deci- sionTreeClassifier, GaussianNB. Note that the predictions of the ensemble method are made based on plurality vote.

• Report the prediction accuracy over an arbitrarily selected test set.

• Compare the performance of your model with sklearn library function 
VotingClassifier.


In [0]:
#logit reg is recall earlier, test/train split also earlier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB

#log reg
model_log = clf.fit(train.iloc[:,0:10], train['quality'])

#naive bayes
clf_nb = GaussianNB()
model_nb = clf_nb.fit(train.iloc[:,0:10], train['quality'])

#decision tree
clf_dt = tree.DecisionTreeClassifier()
model_dt = clf_dt.fit(train.iloc[:,0:10], train['quality'])

In [0]:
#predictions
nb_pred = model_nb.predict(X_test)
dt_pred = model_dt.predict(X_test)
log_pred = model_log.predict(X_test)


In [0]:
all_pred = []
for i in range(140):
  all_pred.append([nb_pred[i], dt_pred[i],log_pred[i]])

In [0]:
vote = []
for i in range(140):
  vote.append(stats.mode(all_pred[i])[0][0])

In [0]:
#our accuracy 
sum(vote == Y_test) / len(Y_test)

0.6357142857142857

In [0]:
##Sklearn voting classifier

from sklearn.ensemble import VotingClassifier

clf_all = VotingClassifier(estimators=[('lr', clf), ('dt', clf_dt), ('gnb', clf_nb)], voting='hard')
vote_clf = clf_all.fit(train.iloc[:,0:10], train['quality'])
vote_clf_pred = vote_clf.predict(X_test)

In [0]:
#86 predictions better than each of them on their own, slightly worse than ours
sum(vote_clf_pred == Y_test) / len(Y_test)

0.6142857142857143